# **Data Loading**

In [1]:
'''
An entry or sent looks like ...
SOCCER NN B-NP O
- : O O
JAPAN NNP B-NP B-LOC
GET VB B-VP O
LUCKY NNP B-NP O
WIN NNP I-NP O
, , O O
CHINA NNP B-NP B-PER
IN IN B-PP O
SURPRISE DT B-NP O
DEFEAT NN I-NP O
. . O O
Each mini-batch returns the followings:
words: list of input sents. ["The 26-year-old ...", ...]
x: encoded input sents. [N, T]. int64.
is_heads: list of head markers. [[1, 1, 0, ...], [...]]
tags: list of tags.['O O B-MISC ...', '...']
y: encoded tags. [N, T]. int64
seqlens: list of seqlens. [45, 49, 10, 50, ...]
'''
import numpy as np
import torch
from torch.utils import data
!pip install pytorch-pretrained-bert
from pytorch_pretrained_bert import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
VOCAB = ('<PAD>', 'O', 'I-LOC', 'B-PER', 'I-PER', 'I-ORG', 'I-MISC', 'B-MISC', 'B-LOC', 'B-ORG')
tag2idx = {tag: idx for idx, tag in enumerate(VOCAB)}
idx2tag = {idx: tag for idx, tag in enumerate(VOCAB)}

class NerDataset(data.Dataset):
    def __init__(self, fpath):
        """
        fpath: [train|valid|test].txt
        """
        entries = open(fpath, 'r').read().strip().split("\n\n")
        sents, tags_li = [], [] # list of lists
        for entry in entries:
            words = [line.split()[0] for line in entry.splitlines()]
            tags = ([line.split()[-1] for line in entry.splitlines()])
            sents.append(["[CLS]"] + words + ["[SEP]"])
            tags_li.append(["<PAD>"] + tags + ["<PAD>"])
        self.sents, self.tags_li = sents, tags_li

    def __len__(self):
        return len(self.sents)

    def __getitem__(self, idx):
        words, tags = self.sents[idx], self.tags_li[idx] # words, tags: string list

        # We give credits only to the first piece.
        x, y = [], [] # list of ids
        is_heads = [] # list. 1: the token is the first piece of a word
        for w, t in zip(words, tags):
            tokens = tokenizer.tokenize(w) if w not in ("[CLS]", "[SEP]") else [w]
            xx = tokenizer.convert_tokens_to_ids(tokens)

            is_head = [1] + [0]*(len(tokens) - 1)

            t = [t] + ["<PAD>"] * (len(tokens) - 1)  # <PAD>: no decision
            yy = [tag2idx[each] for each in t]  # (T,)

            x.extend(xx)
            is_heads.extend(is_head)
            y.extend(yy)

        assert len(x)==len(y)==len(is_heads), f"len(x)={len(x)}, len(y)={len(y)}, len(is_heads)={len(is_heads)}"

        # seqlen
        seqlen = len(y)

        # to string
        words = " ".join(words)
        tags = " ".join(tags)
        return words, x, is_heads, tags, y, seqlen


def pad(batch):
    '''Pads to the longest sample'''
    f = lambda x: [sample[x] for sample in batch]
    words = f(0)
    is_heads = f(2)
    tags = f(3)
    seqlens = f(-1)
    maxlen = np.array(seqlens).max()

    f = lambda x, seqlen: [sample[x] + [0] * (seqlen - len(sample[x])) for sample in batch] # 0: <pad>
    x = f(1, maxlen)
    y = f(-2, maxlen)


    f = torch.LongTensor

    return words, f(x), is_heads, tags, f(y), seqlens

# Testing Data Loader

download data

In [2]:
# train='https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/train.txt'
# valid='https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/valid.txt'
# test='https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/test.txt'

# !mkdir conll2003 | wget --show-progress $train && mv train.txt conll2003
# !wget --show-progress $valid && mv valid.txt conll2003
# !wget --show-progress $test && mv test.txt conll2003

mkdir: cannot create directory ‘conll2003’: File exists
--2019-08-16 22:06:09--  https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283420 (3.1M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]   3.13M  8.50MB/s    in 0.4s    

2019-08-16 22:06:09 (8.50 MB/s) - ‘train.txt’ saved [3283420/3283420]

--2019-08-16 22:06:09--  https://raw.githubusercontent.com/Franck-Dernoncourt/NeuroNER/master/neuroner/data/conll2003/en/valid.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [3]:
# import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument("--trainset", type=str, default="conll2003/train.txt")
# parser.add_argument("--validset", type=str, default="conll2003/valid.txt")
# hp = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model = Net(hp.top_rnns, len(VOCAB), device, hp.finetuning).cuda()
# model = nn.DataParallel(model)

train_dataset = NerDataset("conll2003/train.txt")
eval_dataset = NerDataset("conll2003/valid.txt")

**Model**

In [4]:
import torch
import torch.nn as nn
from pytorch_pretrained_bert import BertModel

class Net(nn.Module):
    def __init__(self, top_rnns=False, vocab_size=None, device='cpu', finetuning=False):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased')

        self.top_rnns=top_rnns
        if top_rnns:
            self.rnn = nn.LSTM(bidirectional=True, num_layers=2, input_size=768, hidden_size=768//2, batch_first=True)
        self.fc = nn.Linear(768, vocab_size)

        self.device = device
        self.finetuning = finetuning

    def forward(self, x, y, ):
        '''
        x: (N, T). int64
        y: (N, T). int64
        Returns
        enc: (N, T, VOCAB)
        '''
        x = x.to(self.device)
        y = y.to(self.device)

        if self.training and self.finetuning:
            # print("->bert.train()")
            self.bert.train()
            encoded_layers, _ = self.bert(x)
            enc = encoded_layers[-1]
        else:
            self.bert.eval()
            with torch.no_grad():
                encoded_layers, _ = self.bert(x)
                enc = encoded_layers[-1]

        if self.top_rnns:
            enc, _ = self.rnn(enc)
        logits = self.fc(enc)
        y_hat = logits.argmax(-1)
        return logits, y, y_hat

# Training

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
# from model import Net
# from data_load import NerDataset, pad, VOCAB, tokenizer, tag2idx, idx2tag
import os
import numpy as np
import argparse

def train(model, iterator, optimizer, criterion):
    model.train()
    for i, batch in enumerate(iterator):
        words, x, is_heads, tags, y, seqlens = batch
        _y = y # for monitoring
        optimizer.zero_grad()
        logits, y, _ = model(x, y) # logits: (N, T, VOCAB), y: (N, T)

        logits = logits.view(-1, logits.shape[-1]) # (N*T, VOCAB)
        y = y.view(-1)  # (N*T,)

        loss = criterion(logits, y)
        loss.backward()

        optimizer.step()

        if i==0:
            print("=====sanity check======")
            print("words:", words[0])
            print("x:", x.cpu().numpy()[0][:seqlens[0]])
            print("tokens:", tokenizer.convert_ids_to_tokens(x.cpu().numpy()[0])[:seqlens[0]])
            print("is_heads:", is_heads[0])
            print("y:", _y.cpu().numpy()[0][:seqlens[0]])
            print("tags:", tags[0])
            print("seqlen:", seqlens[0])
            print("=======================")


        if i%10==0: # monitoring
            print(f"step: {i}, loss: {loss.item()}")

def eval(model, iterator, f):
    model.eval()

    Words, Is_heads, Tags, Y, Y_hat = [], [], [], [], []
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            words, x, is_heads, tags, y, seqlens = batch

            _, _, y_hat = model(x, y)  # y_hat: (N, T)

            Words.extend(words)
            Is_heads.extend(is_heads)
            Tags.extend(tags)
            Y.extend(y.numpy().tolist())
            Y_hat.extend(y_hat.cpu().numpy().tolist())

    ## gets results and save
    with open("temp", 'w') as fout:
        for words, is_heads, tags, y_hat in zip(Words, Is_heads, Tags, Y_hat):
            y_hat = [hat for head, hat in zip(is_heads, y_hat) if head == 1]
            preds = [idx2tag[hat] for hat in y_hat]
            assert len(preds)==len(words.split())==len(tags.split())
            for w, t, p in zip(words.split()[1:-1], tags.split()[1:-1], preds[1:-1]):
                fout.write(f"{w} {t} {p}\n")
            fout.write("\n")

    ## calc metric
    y_true =  np.array([tag2idx[line.split()[1]] for line in open("temp", 'r').read().splitlines() if len(line) > 0])
    y_pred =  np.array([tag2idx[line.split()[2]] for line in open("temp", 'r').read().splitlines() if len(line) > 0])

    num_proposed = len(y_pred[y_pred>1])
    num_correct = (np.logical_and(y_true==y_pred, y_true>1)).astype(np.int).sum()
    num_gold = len(y_true[y_true>1])

    print(f"num_proposed:{num_proposed}")
    print(f"num_correct:{num_correct}")
    print(f"num_gold:{num_gold}")
    try:
        precision = num_correct / num_proposed
    except ZeroDivisionError:
        precision = 1.0

    try:
        recall = num_correct / num_gold
    except ZeroDivisionError:
        recall = 1.0

    try:
        f1 = 2*precision*recall / (precision + recall)
    except ZeroDivisionError:
        if precision*recall==0:
            f1=1.0
        else:
            f1=0

    final = f + ".P%.2f_R%.2f_F%.2f" %(precision, recall, f1)
    with open(final, 'w') as fout:
        result = open("temp", "r").read()
        fout.write(f"{result}\n")

        fout.write(f"precision={precision}\n")
        fout.write(f"recall={recall}\n")
        fout.write(f"f1={f1}\n")

    os.remove("temp")

    print("precision=%.2f"%precision)
    print("recall=%.2f"%recall)
    print("f1=%.2f"%f1)
    return precision, recall, f1

# if __name__=="__main__":
if True:
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--batch_size", type=int, default=128)
#     parser.add_argument("--lr", type=float, default=0.0001)
#     parser.add_argument("--n_epochs", type=int, default=30)
#     parser.add_argument("--finetuning", dest="finetuning", action="store_true")
#     parser.add_argument("--top_rnns", dest="top_rnns", action="store_true")
#     parser.add_argument("--logdir", type=str, default="checkpoints/01")
#     parser.add_argument("--trainset", type=str, default="conll2003/train.txt")
#     parser.add_argument("--validset", type=str, default="conll2003/valid.txt")
#     hp = parser.parse_args()
    torch.cuda.reset_max_memory_allocated(device=None)
    print(torch.cuda.max_memory_allocated(device=None))
    from inspect import currentframe, getsourcelines, getframeinfo
    frameinfo = getframeinfo(currentframe())
    framelines = getsourcelines(currentframe())
    [print(line[:-1]) for line in framelines[0][frameinfo.lineno+2:frameinfo.lineno+7]]
    batch_size = 16
    lr = 0.0001
    n_epochs = 30
    finetuning = True
    top_rnns = False
    logdir = "checkpoints/01"
    trainset = "conll2003/train.txt"
    validset = "conll2003/valid.txt"
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
#     device = 'cpu'

#     model = Net(hp.top_rnns, len(VOCAB), device, hp.finetuning).cuda()
#     model = Net(top_rnns=top_rnns, vocab_size=len(VOCAB), device=device, finetuning=finetuning).cuda()
    model = Net(top_rnns=top_rnns, vocab_size=len(VOCAB), device=device, finetuning=finetuning)

    model = nn.DataParallel(model)
    print(torch.cuda.max_memory_allocated(device=None))
    train_dataset = NerDataset(trainset)
    eval_dataset = NerDataset(validset)

    train_iter = data.DataLoader(dataset=train_dataset,
                                 batch_size=batch_size,
                                 shuffle=True,
                                 num_workers=4,
                                 collate_fn=pad)
    eval_iter = data.DataLoader(dataset=eval_dataset,
                                 batch_size=batch_size,
                                 shuffle=False,
                                 num_workers=4,
                                 collate_fn=pad)

    optimizer = optim.Adam(model.parameters(), lr = lr)
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    print(len(train_iter))
    for epoch in range(1, n_epochs+1):
        try:
            train(model, train_iter, optimizer, criterion)
        except:
            pass
        print(f"=========eval at epoch={epoch}=========")
        if not os.path.exists(logdir): os.makedirs(logdir)
        fname = os.path.join(logdir, str(epoch))
        precision, recall, f1 = eval(model, eval_iter, fname)

        torch.save(model.state_dict(), f"{fname}.pt")
        print(f"weights were saved to {fname}.pt")

0
    batch_size = 16
    lr = 0.0001
    n_epochs = 30
    finetuning = True
    top_rnns = False
433796608
937
=====sanity check======
words: [CLS] DUBAI 1996-08-22 [SEP]
x: [ 101  141 2591 8215 2240 1820  118 4775  118 1659  102]
tokens: ['[CLS]', 'D', '##U', '##BA', '##I', '1996', '-', '08', '-', '22', '[SEP]']
is_heads: [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
y: [0 8 0 0 0 1 0 0 0 0 0]
tags: <PAD> B-LOC O <PAD>
seqlen: 11
step: 0, loss: 2.2521984577178955
step: 10, loss: 0.436301052570343
step: 20, loss: 0.3271217346191406
step: 30, loss: 0.09676904231309891
step: 40, loss: 0.10202603042125702
step: 50, loss: 0.23763076961040497
step: 60, loss: 0.23879751563072205
step: 70, loss: 0.15176692605018616
step: 80, loss: 0.06664877384901047
step: 90, loss: 0.038849838078022
step: 100, loss: 0.1720307469367981
step: 110, loss: 0.1923402100801468
step: 120, loss: 0.09545484930276871
step: 130, loss: 0.08768244087696075
step: 140, loss: 0.24690145254135132
step: 150, loss: 0.11757929623126984
st

step: 340, loss: 0.0661143958568573
step: 350, loss: 0.00867249071598053
step: 360, loss: 0.006173949223011732
step: 370, loss: 0.05764853209257126
step: 380, loss: 0.02455427683889866
step: 390, loss: 0.06349090486764908
step: 400, loss: 0.06947939842939377
step: 410, loss: 0.03597579151391983
step: 420, loss: 0.04264821857213974
step: 430, loss: 0.02358974888920784
step: 440, loss: 0.003530247835442424
step: 450, loss: 0.055052921175956726
step: 460, loss: 0.04485927149653435
step: 470, loss: 0.05754230543971062
step: 480, loss: 0.033246997743844986
step: 490, loss: 0.006853921338915825
step: 500, loss: 0.026338182389736176
step: 510, loss: 0.11784500628709793
step: 520, loss: 0.018559681251645088
step: 530, loss: 0.04403621703386307
step: 540, loss: 0.00773985730484128
step: 550, loss: 0.06577816605567932
step: 560, loss: 0.029047703370451927
step: 570, loss: 0.006913188844919205
step: 580, loss: 0.03171537071466446
step: 590, loss: 0.014535246416926384
step: 600, loss: 0.0185308270

step: 350, loss: 0.008036043494939804
step: 360, loss: 0.005333912093192339
step: 370, loss: 0.058056894689798355
step: 380, loss: 0.03121345303952694
step: 390, loss: 0.002361691789701581
step: 400, loss: 0.06225937232375145
step: 410, loss: 0.01968599669635296
step: 420, loss: 0.0015107366489246488
step: 430, loss: 0.03909780830144882
step: 440, loss: 0.008962721563875675
step: 450, loss: 0.01752508245408535
step: 460, loss: 0.0007687830366194248
step: 470, loss: 0.01131421234458685
step: 480, loss: 0.03618811070919037
step: 490, loss: 0.011290294118225574
step: 500, loss: 0.07933424413204193
step: 510, loss: 0.0502014085650444
step: 520, loss: 0.006985587999224663
step: 530, loss: 0.0060341269709169865
step: 540, loss: 0.008227941580116749
step: 550, loss: 0.0008974510128609836
step: 560, loss: 0.010326836258172989
step: 570, loss: 0.10052009671926498
step: 580, loss: 0.059629887342453
step: 590, loss: 0.02008405700325966
step: 600, loss: 0.010087025351822376
step: 610, loss: 0.0034

step: 190, loss: 0.041838422417640686
step: 200, loss: 0.0033136452548205853
step: 210, loss: 0.0026937646325677633
step: 220, loss: 0.0010001322953030467
step: 230, loss: 0.01746082492172718
step: 240, loss: 0.007711085025221109
step: 250, loss: 0.045195117592811584
step: 260, loss: 0.008755709044635296
step: 270, loss: 0.002219368238002062
step: 280, loss: 0.0006498199072666466
step: 290, loss: 0.009938313625752926
step: 300, loss: 0.058911170810461044
step: 310, loss: 0.01855437271296978
step: 320, loss: 0.0016094333259388804
step: 330, loss: 0.06528449058532715
step: 340, loss: 0.0014535118825733662
step: 350, loss: 0.000432726665167138
step: 360, loss: 0.0005195228150114417
step: 370, loss: 0.04566948488354683
step: 380, loss: 0.005394765175879002
step: 390, loss: 0.03447425737977028
step: 400, loss: 0.03043360635638237
step: 410, loss: 0.0025220243260264397
step: 420, loss: 0.00204424443654716
step: 430, loss: 0.017428189516067505
step: 440, loss: 0.000963857863098383
step: 450, 

step: 420, loss: 0.03136926889419556
step: 430, loss: 0.02324700355529785
step: 440, loss: 0.007621020078659058
step: 450, loss: 0.0662660002708435
step: 460, loss: 0.005316449794918299
step: 470, loss: 0.01739136502146721
step: 480, loss: 0.007939671166241169
step: 490, loss: 0.023798616603016853
step: 500, loss: 0.00520484009757638
step: 510, loss: 0.01281624287366867
step: 520, loss: 0.020562440156936646
step: 530, loss: 0.04854276776313782
step: 540, loss: 0.011619487777352333
step: 550, loss: 0.006516872439533472
step: 560, loss: 0.12083683162927628
step: 570, loss: 0.04584328830242157
step: 580, loss: 0.004932920448482037
step: 590, loss: 0.003931383602321148
step: 600, loss: 0.01666448824107647
step: 610, loss: 0.009469831362366676
=========eval at epoch=13=========
num_proposed:8542
num_correct:7891
num_gold:8603
precision=0.92
recall=0.92
f1=0.92
weights were saved to checkpoints/01/13.pt
=====sanity check======
words: [CLS] CRICKET - ENGLAND NAME SQUAD FOR ONE-DAY INTERNATION

step: 260, loss: 0.019754916429519653
step: 270, loss: 0.06178651005029678
=========eval at epoch=16=========
num_proposed:8529
num_correct:7940
num_gold:8603
precision=0.93
recall=0.92
f1=0.93
weights were saved to checkpoints/01/16.pt
=====sanity check======
words: [CLS] - Iraq 's President Saddam Hussein meets with chairman of the Russian liberal democratic party Vladimir Zhirinovsky . [SEP]
x: [  101   118  5008   112   188  1697 27091 17605  5636  1114  3931  1104
  1103  1938  7691  9327  1710  8591   163 14518  4559 13510   119   102]
tokens: ['[CLS]', '-', 'Iraq', "'", 's', 'President', 'Saddam', 'Hussein', 'meets', 'with', 'chairman', 'of', 'the', 'Russian', 'liberal', 'democratic', 'party', 'Vladimir', 'Z', '##hir', '##ino', '##vsky', '.', '[SEP]']
is_heads: [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1]
y: [0 1 8 1 0 1 3 4 1 1 1 1 1 7 1 1 1 3 4 0 0 0 1 0]
tags: <PAD> O B-LOC O O B-PER I-PER O O O O O B-MISC O O O B-PER I-PER O <PAD>
seqlen: 24
step:

step: 480, loss: 0.007322611287236214
step: 490, loss: 0.007955797016620636
step: 500, loss: 0.002873152494430542
step: 510, loss: 0.019372571259737015
step: 520, loss: 0.00584058603271842
step: 530, loss: 0.039576850831508636
step: 540, loss: 0.0032230867072939873
step: 550, loss: 0.05144813284277916
step: 560, loss: 0.023352615535259247
step: 570, loss: 0.026649853214621544
step: 580, loss: 0.04052000492811203
step: 590, loss: 0.03182521462440491
step: 600, loss: 0.02291063778102398
step: 610, loss: 0.026354433968663216
step: 620, loss: 0.0012637615436688066
step: 630, loss: 0.0021424756851047277
step: 640, loss: 0.007155577186495066
step: 650, loss: 0.0031727852765470743
step: 660, loss: 0.021655945107340813
step: 670, loss: 0.05291448161005974
step: 680, loss: 0.010833803564310074
step: 690, loss: 0.017100568860769272
step: 700, loss: 0.028967885300517082
step: 710, loss: 0.013795576058328152
step: 720, loss: 0.0031148765701800585
step: 730, loss: 0.004249683581292629
step: 740, lo

step: 40, loss: 0.0006269299774430692
step: 50, loss: 0.000541826942935586
step: 60, loss: 0.000659323763102293
step: 70, loss: 0.0004546412965282798
step: 80, loss: 0.00018400134285911918
step: 90, loss: 0.010298162698745728
step: 100, loss: 0.0032770789694041014
step: 110, loss: 0.02961757965385914
step: 120, loss: 0.0007046579848974943
step: 130, loss: 0.0004226487362757325
step: 140, loss: 0.0005355527391657233
step: 150, loss: 0.0013838743325322866
step: 160, loss: 0.0022498841863125563
step: 170, loss: 0.001424493850208819
step: 180, loss: 0.0020515467040240765
step: 190, loss: 0.001409244490787387
step: 200, loss: 0.02602306194603443
step: 210, loss: 0.018150173127651215
step: 220, loss: 0.041245218366384506
step: 230, loss: 0.02516542375087738
step: 240, loss: 0.0009531128453090787
step: 250, loss: 0.01672859489917755
step: 260, loss: 0.11258979886770248
step: 270, loss: 0.00412875646725297
step: 280, loss: 0.0020482358522713184
step: 290, loss: 0.007213279139250517
step: 300, 

KeyboardInterrupt: 

In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)

if cuda torch fails https://pytorch.org/get-started/locally/#cuda-100 

In [ ]:
torch.cuda.max_memory_allocated(device='cuda')

In [ ]:
torch.cuda.reset_max_memory_allocated(device='cuda')

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def pretty_size(size):
    """Pretty prints a torch.Size object"""
    assert(isinstance(size, torch.Size))
    return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
    """Prints a list of the Tensors being tracked by the garbage collector."""
    import gc
    total_size = 0
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if not gpu_only or obj.is_cuda:
                    print("%s:%s%s %s" % (type(obj).__name__, 
                                        " GPU" if obj.is_cuda else "",
                                        " pinned" if obj.is_pinned else "",
                                        pretty_size(obj.size())))
                    total_size += obj.numel()
            elif hasattr(obj, "data") and torch.is_tensor(obj.data):
                if not gpu_only or obj.is_cuda:
                    print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
                                                   type(obj.data).__name__, 
                                                   " GPU" if obj.is_cuda else "",
                                                   " pinned" if obj.data.is_pinned else "",
                                                   " grad" if obj.requires_grad else "", 
                                                   " volatile" if obj.volatile else "",
                                                   pretty_size(obj.data.size())))
                    total_size += obj.data.numel()
                     
        except Exception as e:
            pass        
    print("Total size:", total_size)

In [ ]:
dump_tensors()

In [ ]:
os.path.basename(__file__)

In [ ]:
from inspect import currentframe, getframeinfo, getsourcelines
import traceback
frameinfo = getframeinfo(currentframe())
framelines = getsourcelines(currentframe())
print (frameinfo.filename, frameinfo.lineno)
[print(line) for line in framelines[0][frameinfo.lineno-2:frameinfo.lineno]]